In [4]:
import os
import av
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

r = 4
n = 12

def self_colors():
       hex_colors = ['#FFFF00', '#000000', '#DC143C', '#6A0D91', '#228B22','#D2691E', '#0000CD', '#8B008B', '#C0C0C0', '#E6E6FA','#800000', '#8C8C8C'] #these are the hex code for the colors which we are choosing. We finalised these colors after trying various combinations. There is scope of improvement here.
       return [mcolors.hex2color(hex_color) for hex_color in hex_colors]

def vids_extract(folder_path):
    parent_folder = os.path.abspath(os.path.join(folder_path, os.pardir))

    output_base_folder = os.path.join(parent_folder, "Cartwheel_output")           
    os.makedirs(output_base_folder, exist_ok=True)

    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".avi"):                            #This function is used to extract individual videos from the input files. this is programmed for ".avi" files only. It can be modified based on the input dataset
                video_file = os.path.join(root, file)
                video_name = os.path.splitext(os.path.relpath(video_file, folder_path))[0]
                process_video(video_file, video_name, output_base_folder)

def process_video(video_file, video_name, output_base_folder):

    frames = frames_extracted(video_file)
    if frames is None:
        print(f"Skipping {video_file} due to extraction failure.")
        return
    output_folder = os.path.join(output_base_folder, f'{video_name}_output')
    os.makedirs(output_folder, exist_ok=True)

    for frame_id, frame in enumerate(frames, 1):
        img = np.array(frame.to_image())                                         #the frame received gets converted into an image. This is done to be able to vreate a 4x4 grid on the top of it for division into 16 blocks.
        full_frame_path = os.path.join(output_folder, f'frame_{frame_id}.png')
        cv2.imwrite(full_frame_path, img)
        plt.imshow(img)
        plt.title(f"Frame {frame_id} of {video_name}")
        plt.show()
        colored_histograms_final(frame, r, n, frame_id, output_folder)

def frames_extracted(video_file):
    try:
        container = av.open(video_file)
        frames = [frame for frame in container.decode(video=0)]                                    #This function is to extract the individual frames from the videos
        first_frame = frames[0]                                                                    #Here we have extracted the first frame
        middle_frame = frames[len(frames) // 2]                                                    #Here we have extracted the middle frame
        last_frame = frames[-1]                                                                    #here we extracted the last frame from the video file
        return [first_frame, middle_frame, last_frame]
    except UnicodeDecodeError as e:
          print(f"UnicodeDecodeError: Skipping file {video_file}. Error: {e}")
          return None
    except Exception as e:
          print(f"Error processing file {video_file}. Error: {e}")
          return None

def colored_histograms_final(frame, r, n, frame_id, output_folder):
    try:
        img = np.array(frame.to_image())
    except Exception as e:
        return None

    h, w, _ = img.shape
    cell_h, cell_w = h // r, w // r
    color_names = ['Yellow','black','red','Purple','Green',"Brown","Blue",'Magenta','Silver','Lavender','Maroon','Grey']  #these are colors used for the histogram plot labelling

    for i in range(r):
        for j in range(r):
            cell = img[i * cell_h:(i + 1) * cell_h, j * cell_w:(j + 1) * cell_w]
            r_histogram = cv2.calcHist([cell], [0], None, [n], [0, 256]).flatten()
            g_histogram = cv2.calcHist([cell], [1], None, [n], [0, 256]).flatten()
            b_histogram = cv2.calcHist([cell], [2], None, [n], [0, 256]).flatten()
            r_histogram, g_histogram, b_histogram = r_histogram / r_histogram.sum(), g_histogram / g_histogram.sum(), b_histogram / b_histogram.sum() #this is done to normalise the frequency for the y axis. The no. of pixels in each color are collected and normalised to a scale of 0 to 1.
            histogram_final = [sum(x) for x in zip(r_histogram, g_histogram, b_histogram)]

            plt.figure()
            plt.bar(range(n), histogram_final, color=color_names)
            plt.xticks(range(n), color_names, rotation=90)
            plt.title(f'Combined RGB Histogram for Frame {frame_id} - Part ({i}, {j})')  # we get 16 histograms for each frame.
            plt.xlabel('Colors')
            plt.ylabel('Frequency (normalized)')
            plt.tight_layout()

            histogram_fianl_path = os.path.join(output_folder, f'hist_frame_{frame_id}_part_{i}_{j}.png')
            plt.savefig(histogram_fianl_path)                                                     #We save total of 3 things in the folder: The frame image,The block image(each of the 16 frame blocks made by the 4x4 grid), and a corresponding histogram to each block
            plt.close()
            part_image_path = os.path.join(output_folder, f'frame_{frame_id}_part_{i}_{j}.png')
            cv2.imwrite(part_image_path, cell)

folder_path = 'Target_video/Cartwheel'
vids_extract(folder_path)